# Treinamento

## Inicialização

In [1]:
# Config Inicial
import json
import mlflow.pyfunc
import shutil
import pandas as pd
import requests
from sklearn.metrics import r2_score
import mlflow
from mlflow.tracking import MlflowClient

import os
import signal
import subprocess


### Leitura

In [2]:
# Descomentar, quando quiser carregar para o treinamento, dados originais defasados para primeira execução do treinamento
#shutil.copyfile("dataset/brasil_estado_cidade.csv", "dataset/dados_treinamento.csv")

# Carregar o conjunto de dados
df_treinamento = pd.read_csv("dataset/dados_treinamento.csv", encoding="utf-8")

# Eliminando registros com valores null
df_treinamento.dropna(inplace=True)

# float64
df_treinamento = df_treinamento.astype({col: 'float64' for col in df_treinamento.select_dtypes(include='int').columns})

df_treinamento.head()

,id,property_type,state,region,lat,lon,area_m2,price_brl,city
0,1.0,apartment,Pernambuco,Northeast,-8.134204,-34.906326,72.0,414222.98,Recife
1,2.0,apartment,Pernambuco,Northeast,-8.126664,-34.903924,136.0,848408.53,Recife
2,3.0,apartment,Pernambuco,Northeast,-8.125550,-34.907601,75.0,299438.28,Recife
3,4.0,apartment,Pernambuco,Northeast,-8.120249,-34.895920,187.0,848408.53,Recife
4,5.0,apartment,Pernambuco,Northeast,-8.142666,-34.906906,80.0,464129.36,Recife


In [3]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")

mlflow.set_experiment("ecd15")

<Experiment: artifact_location='/app/mlruns/0', creation_time=1744907315763, experiment_id='0', last_update_time=1744907894136, lifecycle_stage='active', name='ecd15', tags={}>

### Amostragem

In [4]:
# amostra
df_sample = df_treinamento.sample(n=20, random_state=42)

x_features = df_sample.drop(["price_brl"], axis=1)  # Features
y_target = df_sample["price_brl"]  # Variável alvo

df_sample.head()

,id,property_type,state,region,lat,lon,area_m2,price_brl,city
3823,3824.0,apartment,Rio de Janeiro,Southeast,-22.930788,-43.355751,62.0,419213.60,Rio de Janeiro
376,377.0,apartment,Piau,Northeast,-5.064457,-42.792750,60.0,319400.84,Teresina
5473,5474.0,apartment,Rio de Janeiro,Southeast,-22.933163,-43.174660,78.0,1287584.69,Rio de Janeiro
2282,2283.0,apartment,Rio Grande do Sul,South,-30.037640,-51.189133,104.0,726908.65,Porto Alegre
980,981.0,apartment,Rio Grande do Sul,South,-30.080242,-51.225227,60.0,494701.71,Porto Alegre


In [5]:
# Gerando o JSON no formato esperado pelo MLflow Serve
dados = {"instances": x_features.to_dict(orient="records")}
json_data = json.dumps(dados)

In [6]:
previsao = requests.post(
    "http://localhost:5000/invocations",
    headers={"Content-Type": "application/json"},
    data=json_data
)
print("HTTP STATUS", previsao.status_code)

HTTP STATUS 200


In [7]:
previsao.text

'{"predictions": [348099.1875, 314764.6875, 963706.625, 730391.0, 330280.34375, 604923.6875, 1018210.0625, 668755.375, 1182253.375, 1222384.375, 798778.8125, 1070625.5, 355035.8125, 863029.625, 542877.5625, 1041795.375, 491103.1875, 373186.9375, 512069.875, 270661.0625]}'

### Carregar e identificar o melhor modelo treinado

In [8]:

def carregar_modelo_com_melhor_r2(experimento_nome):
    client = MlflowClient()

    # Pega o experimento
    experimento = client.get_experiment_by_name(experimento_nome)
    if not experimento:
        raise ValueError(f"Experimento '{experimento_nome}' não encontrado.")
    
    experiment_id = experimento.experiment_id

    melhor_modelo = {
        "r2": float('-inf'),
        "model_uri": None,
        "nome_modelo": None,
        "versao_modelo": None,
        "run_id": None
    }

    modelo_api = {
        "r2": float('-inf'),
        "model_uri": None,
        "nome_modelo": None,
        "versao_modelo": None,
        "run_id": None
    }

    # Pega todas as runs ordenadas por R² (decrescente)
    runs = client.search_runs(
        experiment_ids=[experiment_id],
        order_by=["metrics.r2 DESC"]
    )

    modelos_registrados = client.search_registered_models()

    

    for modelo in modelos_registrados:
        
        nome_modelo = modelo.name

        for versao in modelo.latest_versions:
            # Busca informações completas da versão
            versoes = client.search_model_versions(f"name='{nome_modelo}'")
            for v in versoes:
                run = client.get_run(v.run_id)

                # Verifica se pertence ao experimento desejado
                if run.info.experiment_id != experiment_id:
                    continue

                r2 = run.data.metrics.get("r2")

                source_path = v.source
                artifact_path = source_path.split("/")[-1]
                model_uri = f"runs:/{v.run_id}/{artifact_path}"

                print(f"Modelo '{modelo.name}' versão {v.version} da run {v.run_id} com R² = {r2}")

                if v.current_stage == "Production":
                    modelo_api.update({
                        "r2": r2,
                        "model_uri": model_uri,
                        "nome_modelo": nome_modelo,
                        "versao_modelo": v.version,
                        "run_id": v.run_id
                    })
                
                if r2 is not None and r2 > melhor_modelo["r2"] and v.current_stage != "Archived":
                    melhor_modelo.update({
                        "r2": r2,
                        "model_uri": model_uri,
                        "nome_modelo": nome_modelo,
                        "versao_modelo": v.version,
                        "run_id": v.run_id
                    })
    
    
    print(f"Melhor modelo '{melhor_modelo['nome_modelo']}' versão {melhor_modelo['versao_modelo']} da run {melhor_modelo['run_id']} com R² = {melhor_modelo['r2']:.4f}")

    print(f"Modelo API '{modelo_api['nome_modelo']}' versão {modelo_api['versao_modelo']} da run {modelo_api['run_id']} com R² = {modelo_api['r2']:.4f}")

    modelo_carregado = mlflow.pyfunc.load_model(melhor_modelo["model_uri"])
    
    return modelo_carregado, melhor_modelo, modelo_api
    

# Carregando modelo com melhor R2 
loaded_model, melhor_modelo, modelo_api = carregar_modelo_com_melhor_r2("ecd15")

if not melhor_modelo["model_uri"]:
    print("Nenhum modelo em 'Staging' encontrado para esse experimento.")

existe_modelo_api = True
if not modelo_api["model_uri"]:
    existe_modelo_api = False
    print("Nenhum modelo em 'Production' encontrado para esse experimento.")

melhor_modelo, modelo_api


Modelo 'decision_tree_model' versão 9 da run 01390f347f15453aac15aca4f0dd63ff com R² = 0.44289446310468705
Modelo 'decision_tree_model' versão 8 da run e2bfe8f204a6443b948b43d1d1e9fb3e com R² = 0.4501405374386255
Modelo 'decision_tree_model' versão 7 da run 3b26914ee43a42a1b555950bab92eff0 com R² = 0.4662427582353449
Modelo 'decision_tree_model' versão 6 da run 87b82fd6df43456cbac3f5b23fc1bff6 com R² = 0.4342869398037643
Modelo 'decision_tree_model' versão 5 da run b6f5c3ba42264a0eb2a1f59bf9583e13 com R² = 0.4688968380333659
Modelo 'decision_tree_model' versão 4 da run 3acdfe766be546bca53899f8519cfe6b com R² = 0.4607357460145043
Modelo 'decision_tree_model' versão 3 da run 2c1c0982f0934b0eb0200e07dbd3daa4 com R² = 0.47475884579582217
Modelo 'decision_tree_model' versão 2 da run 9ebb52752ae84be58ef8aa5eacad1477 com R² = 0.4839914622045445
Modelo 'decision_tree_model' versão 1 da run ab47ff20b3ad490ea4e8f847603a667c com R² = 0.4643648515387715
Modelo 'gradient_boosting_model' versão 9 da

({'r2': 0.7165948434149507,
  'model_uri': 'runs:/8cc1b3f756e34b03bbc93c19aace7a92/xgboost_model',
  'nome_modelo': 'xgboost_model',
  'versao_modelo': 7,
  'run_id': '8cc1b3f756e34b03bbc93c19aace7a92'},
 {'r2': 0.7165948434149507,
  'model_uri': 'runs:/8cc1b3f756e34b03bbc93c19aace7a92/xgboost_model',
  'nome_modelo': 'xgboost_model',
  'versao_modelo': 7,
  'run_id': '8cc1b3f756e34b03bbc93c19aace7a92'})

## Predição de melhor modelo carregado x modelo API

In [9]:
# Fazer a previsão
predictions = loaded_model.predict(x_features)
# Adicionar as previsões ao DataFrame
    
df_sample["predicao_melhor_modelo"] = predictions

# converter previsões para o mesmo tipo da variável alvo
df_sample["predicao_melhor_modelo"] = df_sample["predicao_melhor_modelo"].astype(float).round(2)

colunas_exibidas = ["price_brl", "predicao_melhor_modelo"]

if existe_modelo_api:
    previsao_json = json.loads(previsao.text)

    # Extrair as previsões do JSON e adicioná-las ao DataFrame
    df_sample["predicao_modelo_api"] = [round(pred, 2) for pred in previsao_json["predictions"]]

    colunas_exibidas = ["price_brl", "predicao_modelo_api", "predicao_melhor_modelo"]

df_sample[colunas_exibidas]

,price_brl,predicao_modelo_api,predicao_melhor_modelo
3823,419213.60,348099.19,348099.19
376,319400.84,314764.69,314764.69
5473,1287584.69,963706.62,963706.62
2282,726908.65,730391.00,730391.00
980,494701.71,330280.34,330280.34
10954,558951.47,604923.69,604923.69
6607,993137.02,1018210.06,1018210.06
1212,678726.79,668755.38,668755.38
6408,1362953.71,1182253.38,1182253.38
11009,1274609.05,1222384.38,1222384.38


### Comparação R² Score dos modelos

In [10]:
# Calcular R²
r2_melhor_modelo = r2_score(df_sample["price_brl"], df_sample["predicao_melhor_modelo"])

if existe_modelo_api:
    r2_api = r2_score(df_sample["price_brl"], df_sample["predicao_melhor_modelo"])
    print(f"R² API: {r2_melhor_modelo:.4f} R² Melhor Modelo: {r2_api:.4f}")
else:
    print(f"R² Melhor Modelo: {r2_melhor_modelo:.4f}")

R² API: 0.7869 R² Melhor Modelo: 0.7869


### Promover melhor modelo para Produção

In [ ]:
def parar_modelo_mlflow1():
    try:
        resultado = subprocess.check_output("lsof -i :5000 -sTCP:LISTEN -nP", shell=True)
        linhas = resultado.decode().strip().split("\n")[1:]  # ignora cabeçalho
        for linha in linhas:
            if "mlflow" in linha.lower():
                pid = int(linha.split()[1])
                os.kill(pid, signal.SIGTERM)
                print(f"Servidor MLflow (PID {pid}) parado com sucesso.")
            else:
                print("⚠️ Porta 5000 ocupada por outro processo que não é o MLflow. Ignorado.")
    except subprocess.CalledProcessError:
        print("Nenhum processo ouvindo na porta 5000 foi encontrado.")

def promover_modelo_para_producao(modelo):
    client = MlflowClient()

    nome_modelo = modelo["nome_modelo"]
    versao_modelo = modelo["versao_modelo"]

    # Promover o modelo selecionado
    client.transition_model_version_stage(
        name=nome_modelo,
        version=versao_modelo,
        stage="Production",
        archive_existing_versions=True  # arquiva automaticamente os outros em Production
    )
    print(f"Modelo '{nome_modelo}' versão {versao_modelo} promovido para 'Production'.")

    # Agora vamos arquivar os que ainda estão em Staging
    todos_modelos = client.search_model_versions("name LIKE '%_model'")
    modelos_ativos = [v for v in todos_modelos if v.current_stage != "Archived"]
    
    for v in modelos_ativos:
        if v.version != versao_modelo and v.name != nome_modelo:
            client.transition_model_version_stage(
                name=nome_modelo,
                version=v.version,
                stage="Archived"
            )
            print(f"Modelo { v.name } Versão {v.version} arquivado.")

def servir_melhor_modelo(experimento_nome, modelo):
    client = MlflowClient()
    
    # Parar modelo atual
    os.system("bash parar_mlflow_serve.sh")

    model_uri = modelo["model_uri"]

    promover_modelo_para_producao(modelo)

    # Reinicializar api com novo modelo
    os.system(f"bash reiniciar_mlflow_serve.sh {model_uri}")

servir_melhor_modelo("ecd15", melhor_modelo)

O processo na porta 5000 foi encerrado com sucesso.
Modelo 'xgboost_model' versão 7 promovido para 'Production'.


/tmp/ipykernel_271/2912602800.py:22: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(
/tmp/ipykernel_271/2912602800.py:36: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


Modelo gradient_boosting_model Versão 9 arquivado.
Modelo random_forest_model Versão 9 arquivado.
Modelo linear_regression_model Versão 9 arquivado.
Modelo decision_tree_model Versão 9 arquivado.
Modelo gradient_boosting_model Versão 8 arquivado.
Modelo random_forest_model Versão 8 arquivado.
Modelo linear_regression_model Versão 8 arquivado.
Modelo decision_tree_model Versão 8 arquivado.
Modelo gradient_boosting_model Versão 6 arquivado.
Modelo random_forest_model Versão 6 arquivado.
Modelo linear_regression_model Versão 6 arquivado.
Modelo decision_tree_model Versão 6 arquivado.
Modelo gradient_boosting_model Versão 5 arquivado.
Modelo random_forest_model Versão 5 arquivado.
Modelo linear_regression_model Versão 5 arquivado.
Modelo decision_tree_model Versão 5 arquivado.
Modelo gradient_boosting_model Versão 4 arquivado.
Modelo random_forest_model Versão 4 arquivado.
Modelo linear_regression_model Versão 4 arquivado.
Modelo decision_tree_model Versão 4 arquivado.
Modelo gradient_boos

2025/04/17 17:17:03 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2025/04/17 17:17:03 INFO mlflow.pyfunc.backend: === Running command 'exec uvicorn --host 0.0.0.0 --port 5000 --workers 1 mlflow.pyfunc.scoring_server.app:app'
INFO:     Started server process [674]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:5000 (Press CTRL+C to quit)


INFO:     172.18.0.1:43928 - "GET / HTTP/1.1" 404 Not Found


...

Modelo decision_tree_model Versão 5 arquivado.
Modelo gradient_boosting_model Versão 4 arquivado.
Modelo linear_regression_model Versão 4 arquivado.


Modelo decision_tree_model Versão 4 arquivado.
Modelo gradient_boosting_model Versão 3 arquivado.
Modelo linear_regression_model Versão 3 arquivado.


Modelo decision_tree_model Versão 3 arquivado.
Modelo gradient_boosting_model Versão 2 arquivado.
Modelo linear_regression_model Versão 2 arquivado.


Modelo decision_tree_model Versão 2 arquivado.
Modelo gradient_boosting_model Versão 1 arquivado.
Modelo linear_regression_model Versão 1 arquivado.


Modelo decision_tree_model Versão 1 arquivado.
